# Barrett Thesis Demo: Generating and Classifying Lyrics with LSTM Networks

In [0]:
from keras.models import model_from_json
import pickle
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


# Generator Demo

In [0]:
with open('drive/My Drive/Colab Notebooks/data/t_swift_model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

model.load_weights('drive/My Drive/Colab Notebooks/data/final_tswift_model_weights.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 24, 50)            122300    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               92160     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2446)              315534    
Total params: 529,994
Trainable params: 529,994
Non-trainable params: 0
_________________________________________________________________


Generator Function

In [0]:
with open('drive/My Drive/Colab Notebooks/data/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

max_len = 25
ind_to_word = dict((v,k) for k,v in tokenizer.word_index.items())


def generate_text(seed_text, num_of_words_to_gen):

    for j in range(num_of_words_to_gen):
        #format token as a model input
        seed_token = tokenizer.texts_to_sequences([seed_text])[0]
        seed_token = pad_sequences([seed_token], maxlen=max_len-1, padding='pre')
        #predict
        predicted = model.predict_classes(seed_token, verbose=0)
        output_word = ind_to_word[predicted[0]]

        seed_text = seed_text + " " + output_word
        if j % 10 == 0 and j != 0:
          seed_text = seed_text + "\n"
    print('\n')
    return seed_text

In [0]:
temp = generate_text('arizona star', 100)
print(temp)



arizona star i was enchanted to meet you when i was drowning you're
 still in his pocket open open for the door for
 you come right on us life ain't like a little
 same times if i had known now i don't know
 what you would be little mind caught up like this
 time why you are you gave we in the very
 were ready dreaming of me this alright mad
 eye year pretty every life waiting plaid alright hunters least
 thought you pushed like small did could be mad love
 me this mad mad mad this mad strange onto


# Classifier **Demo**

In [0]:
with open('drive/My Drive/Colab Notebooks/data/classifier_model_architecture.json', 'r') as f:
  c_model = model_from_json(f.read())

c_model.load_weights('drive/My Drive/Colab Notebooks/data/final_classifier_model_weights.h5')


with open('drive/My Drive/Colab Notebooks/data//classifier_tokenizer.pickle', 'rb') as handle:
    tok = pickle.load(handle)
  
  
  
one_hot = {'Pop':[1,0,0,0], 'Hip-Hop':[0,1,0,0], 'Metal': [0,0,1,0], 'Rock':[0,0,0,1]}

c_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_4 (CuDNNLSTM)     (None, 128)               67072     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 516       
_________________________________________________________________
activation_4 (Activation)    (None, 4)                 0         
Total params: 67,588
Trainable params: 67,588
Non-trainable params: 0
_________________________________________________________________


In [0]:
song = input()

song = song.lower()
song = song.replace('\n', ' ')
song = song.split(' ')
song = [song]

#break into seqs
sequences = tok.texts_to_sequences(song)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=8196)
sequences_matrix = sequences_matrix.reshape(1, 8196, 1)

#predict
output = c_model.predict(sequences_matrix)
print(output)
#jank solution to output genre name
song = [0,0,0,0]
max_val = np.argmax(output)
song[max_val] = 1
for k,v in one_hot.items():
  if song == v:
    print(k)
    break


Thought I'd end up with Sean But he wasn't a match Wrote some songs about Ricky Now I listen and laugh Even almost got married And for Pete, I'm so thankful Wish I could say, "Thank you" to Malcolm 'Cause he was an angel One taught me love One taught me patience And one taught me pain Now, I'm so amazing I've loved and I've lost But that's not what I see So, look what I got Look what you taught me And for that, I say Thank you, next (next) Thank you, next (next) Thank you, next I'm so fuckin' grateful for my ex Thank you, next (next) Thank you, next (next) Thank you, next (next) I'm so fuckin' Spend more time with my friends I ain't worried 'bout nothin' Plus, I met someone else We havin' better discussions I know they say I move on too fast But this one gon' last 'Cause her name is Ari And I'm so good with that (so good with that) She taught me love (love) She taught me patience (patience) How she handles pain (pain) That shit's amazing (yeah, she's amazing) I've loved and I've lost (